In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

def preprocess_air_quality_data(file_path):
    # Load data
    df = pd.read_csv(file_path, low_memory=False)
    df.drop(columns=['City'], inplace=True)
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=['float64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
    
    # Process datetime and create time features
    df['Datetime'] = pd.to_datetime(df['Datetime'], format='mixed', errors='coerce')
    df['Year'] = df['Datetime'].dt.year
    df['Month'] = df['Datetime'].dt.month
    df['Day'] = df['Datetime'].dt.day
    df['Hour'] = df['Datetime'].dt.hour
    
    # Categorize PM2.5 values
    bins = [0, 12, 35.4, 55.4, 150.4, 250.4, float('inf')]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['PM2.5_Category'] = pd.cut(df['PM2.5'], bins=bins, labels=labels)
    
    # Perform undersampling
    return df


# For each dataset
df = preprocess_air_quality_data('../Data/Patna_data.csv')

# Print sizes to see the reduction
print("Original sizes:", len(df))

Original sizes: 44554


In [4]:
df.head()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Year,Month,Day,Hour,PM2.5_Category
0,2015-06-01 15:00:00,121.156521,126.977728,2.11,7.16,8.01,18.36667,1.01,2.72,8.47,0.0,0.71,0.0,237.929606,NaN,2015,6,1,15,Unhealthy
1,2015-06-01 16:00:00,121.156521,126.977728,26.47,27.17,53.65,18.36667,1.31,2.01,8.92,0.0,0.08,0.0,237.929606,NaN,2015,6,1,16,Unhealthy
2,2015-06-01 17:00:00,121.156521,126.977728,15.55,29.62,45.17,18.36667,1.43,1.12,6.97,0.0,0.00,0.0,237.929606,NaN,2015,6,1,17,Unhealthy
3,2015-06-01 18:00:00,121.156521,126.977728,7.68,27.13,34.81,18.36667,1.62,1.54,10.95,0.0,0.00,0.0,237.929606,NaN,2015,6,1,18,Unhealthy
4,2015-06-01 19:00:00,121.156521,126.977728,28.02,33.30,61.31,18.36667,2.25,2.36,9.01,0.0,0.63,0.0,237.929606,NaN,2015,6,1,19,Unhealthy


In [5]:
df.describe()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,Year,Month,Day,Hour
count,44554,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000
mean,2017-12-15 19:30:00,121.156521,126.977728,31.332888,37.939637,44.569077,18.366670,1.552188,23.472227,38.807114,1.761845,11.254970,5.767587,237.929606,2017.456996,6.512838,15.735916,11.501257
min,2015-06-01 15:00:00,0.250000,13.640000,0.010000,0.010000,0.000000,0.080000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,38.000000,2015.000000,1.000000,1.000000,0.000000
25%,2016-09-07 17:15:00,57.000000,126.977728,8.040000,17.360000,15.622500,18.366670,0.840000,5.410000,12.480000,0.300000,1.410000,0.600000,156.000000,2016.000000,4.000000,8.000000,6.000000
50%,2017-12-15 19:30:00,121.156521,126.977728,22.550000,34.510000,40.755000,18.366670,1.370000,23.472227,38.807114,1.390000,6.070000,3.280000,237.929606,2017.000000,6.000000,16.000000,12.000000
75%,2019-03-24 21:45:00,123.990000,126.977728,31.332888,40.050000,49.260000,18.366670,1.560000,36.320000,43.610000,1.830000,11.254970,5.767587,309.000000,2019.000000,9.000000,23.000000,18.000000
max,2020-07-01 00:00:00,994.000000,487.700000,494.090000,466.050000,496.000000,64.130000,30.000000,199.770000,199.510000,404.060000,292.160000,319.950000,655.000000,2020.000000,12.000000,31.000000,23.000000
std,NaN,102.885708,23.182625,39.306657,33.544288,44.683502,3.817473,1.463131,19.641528,33.836939,3.241802,14.471385,10.280883,105.892997,1.521099,3.422432,8.797692,6.922620


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44554 entries, 0 to 44553
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Datetime        44554 non-null  datetime64[ns]
 1   PM2.5           44554 non-null  float64       
 2   PM10            44554 non-null  float64       
 3   NO              44554 non-null  float64       
 4   NO2             44554 non-null  float64       
 5   NOx             44554 non-null  float64       
 6   NH3             44554 non-null  float64       
 7   CO              44554 non-null  float64       
 8   SO2             44554 non-null  float64       
 9   O3              44554 non-null  float64       
 10  Benzene         44554 non-null  float64       
 11  Toluene         44554 non-null  float64       
 12  Xylene          44554 non-null  float64       
 13  AQI             44554 non-null  float64       
 14  AQI_Bucket      32318 non-null  object        
 15  Ye

In [7]:
df.isna().sum()

Datetime              0
PM2.5                 0
PM10                  0
NO                    0
NO2                   0
NOx                   0
NH3                   0
CO                    0
SO2                   0
O3                    0
Benzene               0
Toluene               0
Xylene                0
AQI                   0
AQI_Bucket        12236
Year                  0
Month                 0
Day                   0
Hour                  0
PM2.5_Category        0
dtype: int64

In [8]:
df_tmp = df.copy()

In [9]:
df_tmp.drop(['AQI_Bucket', 'Datetime'], axis=1, inplace=True)
df_tmp.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,Year,Month,Day,Hour,PM2.5_Category
0,121.156521,126.977728,2.11,7.16,8.01,18.36667,1.01,2.72,8.47,0.0,0.71,0.0,237.929606,2015,6,1,15,Unhealthy
1,121.156521,126.977728,26.47,27.17,53.65,18.36667,1.31,2.01,8.92,0.0,0.08,0.0,237.929606,2015,6,1,16,Unhealthy
2,121.156521,126.977728,15.55,29.62,45.17,18.36667,1.43,1.12,6.97,0.0,0.00,0.0,237.929606,2015,6,1,17,Unhealthy
3,121.156521,126.977728,7.68,27.13,34.81,18.36667,1.62,1.54,10.95,0.0,0.00,0.0,237.929606,2015,6,1,18,Unhealthy
4,121.156521,126.977728,28.02,33.30,61.31,18.36667,2.25,2.36,9.01,0.0,0.63,0.0,237.929606,2015,6,1,19,Unhealthy


In [10]:
df_tmp['PM2.5_Category'] = df_tmp['PM2.5_Category'].astype("category").cat.as_ordered()

In [11]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44554 entries, 0 to 44553
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   PM2.5           44554 non-null  float64 
 1   PM10            44554 non-null  float64 
 2   NO              44554 non-null  float64 
 3   NO2             44554 non-null  float64 
 4   NOx             44554 non-null  float64 
 5   NH3             44554 non-null  float64 
 6   CO              44554 non-null  float64 
 7   SO2             44554 non-null  float64 
 8   O3              44554 non-null  float64 
 9   Benzene         44554 non-null  float64 
 10  Toluene         44554 non-null  float64 
 11  Xylene          44554 non-null  float64 
 12  AQI             44554 non-null  float64 
 13  Year            44554 non-null  int32   
 14  Month           44554 non-null  int32   
 15  Day             44554 non-null  int32   
 16  Hour            44554 non-null  int32   
 17  PM2.5_Catego

In [12]:
df_tmp['PM2.5_Category'].cat.categories

Index(['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy',
       'Very Unhealthy', 'Hazardous'],
      dtype='object')

In [13]:
df_tmp['PM2.5_Category'].cat.codes

0        3
1        3
2        3
3        3
4        3
        ..
44549    1
44550    1
44551    1
44552    2
44553    2
Length: 44554, dtype: int8

In [14]:
df_tmp.isnull().sum()/len(df_tmp)

PM2.5             0.0
PM10              0.0
NO                0.0
NO2               0.0
NOx               0.0
NH3               0.0
CO                0.0
SO2               0.0
O3                0.0
Benzene           0.0
Toluene           0.0
Xylene            0.0
AQI               0.0
Year              0.0
Month             0.0
Day               0.0
Hour              0.0
PM2.5_Category    0.0
dtype: float64

In [15]:
np.random.seed(42)
from sklearn.model_selection import train_test_split

X = df_tmp.drop("PM2.5", axis=1)
Y = df_tmp["PM2.5"]

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8)

len(X_train), len(X_valid)

X_train.shape, X_valid.shape

((35643, 17), (8911, 17))

In [16]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)


PM10
NO
NO2
NOx
NH3
CO
SO2
O3
Benzene
Toluene
Xylene
AQI
Year
Month
Day
Hour


In [17]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label, "--> has", pd.isnull(content).sum(), "null values")

In [18]:
pd.Categorical(X_train["PM2.5_Category"]).codes

array([3, 3, 3, ..., 3, 3, 3], dtype=int8)

In [19]:
for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        #turn category into number
        X_train[label] = pd.Categorical(content).codes + 1

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35643 entries, 959 to 15795
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PM10            35643 non-null  float64
 1   NO              35643 non-null  float64
 2   NO2             35643 non-null  float64
 3   NOx             35643 non-null  float64
 4   NH3             35643 non-null  float64
 5   CO              35643 non-null  float64
 6   SO2             35643 non-null  float64
 7   O3              35643 non-null  float64
 8   Benzene         35643 non-null  float64
 9   Toluene         35643 non-null  float64
 10  Xylene          35643 non-null  float64
 11  AQI             35643 non-null  float64
 12  Year            35643 non-null  int32  
 13  Month           35643 non-null  int32  
 14  Day             35643 non-null  int32  
 15  Hour            35643 non-null  int32  
 16  PM2.5_Category  35643 non-null  int8   
dtypes: float64(12), int32(4), int8(1)


In [21]:
for label, content in X_valid.items():
    if not pd.api.types.is_numeric_dtype(content):
        X_valid[label] = pd.Categorical(content).codes + 1

In [22]:
X_train.shape, X_valid.shape

((35643, 17), (8911, 17))

In [23]:
%%time
# Lets build a machine learning model
from sklearn.ensemble import RandomForestRegressor

CPU times: total: 0 ns
Wall time: 56.7 ms


In [24]:
np.random.seed(42)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    calculate root mean square log error between prediction and true labels    
    """

    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# create a function to evaluate model on a few different levels

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE" : mean_absolute_error(Y_train,train_preds),
              "valid MAE" : mean_absolute_error(Y_valid, val_preds),
              "Training RMSLE" : rmsle(Y_train,train_preds),
              "valid RMSLE" : rmsle(Y_valid, val_preds),
              "Training R^2": r2_score(Y_train,train_preds),
              "valid R^2" : r2_score(Y_valid, val_preds)}
    
    return scores

In [25]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [26]:
%%time
model.fit(X_train, Y_train)

CPU times: total: 1min 2s
Wall time: 6.92 s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [27]:
show_scores(model)

{'Training MAE': 4.641161245362339,
 'valid MAE': 12.381011118388756,
 'Training RMSLE': 0.059526264455895206,
 'valid RMSLE': 0.152583410112753,
 'Training R^2': 0.985168053194364,
 'valid R^2': 0.9037792674429758}

In [28]:
%%time

# most ideal parameter 

ideal_model = RandomForestRegressor(n_estimators=90,
                                    min_samples_leaf=18,
                                    min_samples_split=3,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42)  # random state so that our reult is reproducable

# FIT THE MODEL

ideal_model.fit(X_train, Y_train)

CPU times: total: 14.2 s
Wall time: 1.65 s


RandomForestRegressor(max_features=0.5, min_samples_leaf=18,
                      min_samples_split=3, n_estimators=90, n_jobs=-1,
                      random_state=42)

In [29]:
show_scores(ideal_model)

{'Training MAE': 12.681179995175158,
 'valid MAE': 14.18387078848996,
 'Training RMSLE': 0.15165447164308343,
 'valid RMSLE': 0.17185201495470181,
 'Training R^2': 0.8972116526854338,
 'valid R^2': 0.8868232706402076}

# Linear Regression

In [30]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,Y_train)

LinearRegression()

In [31]:
y_pred = linreg.predict(X_valid)

In [32]:
from sklearn.model_selection import cross_val_score
print('Cross_val_score', cross_val_score(linreg, X_train, Y_train, cv=10, scoring="r2").mean())
print('r2_score', r2_score(Y_valid, y_pred))
print("valid MAE ", mean_absolute_error(Y_valid, y_pred))

Cross_val_score 0.6599646222572553
r2_score 0.6857024503901419
valid MAE  29.073809431314974
